In [146]:
# !pip install missingno
# !pip install geopy

In [147]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [148]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [149]:
AirbnbBerlin_df = pd.read_csv('/content/drive/My Drive/Airbnb/Airbnb Berlin.csv', index_col=0)
# AirbnbBerlin_df = pd.read_csv('/content/Airbnb Berlin.csv')
# df_2019 = AirbnbBerlin_df[AirbnbBerlin_df['review_date'].astype(str).str[6:] == '19']

# df_2019 = pd.read_csv('/content/drive/My Drive/Airbnb/AirbnbBerlin_2019.csv', index_col=0)
# df_2019 = pd.read_csv('./content/AirbnbBerlin_2019.csv')

In [169]:
df = AirbnbBerlin_df.copy()

# 1. Data Preparation

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456961 entries, 0 to 456960
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Review ID              452805 non-null  float64
 1   review_date            452805 non-null  object 
 2   Reviewer ID            452805 non-null  float64
 3   Reviewer Name          452805 non-null  object 
 4   Comments               452595 non-null  object 
 5   Listing ID             456961 non-null  int64  
 6   Listing URL            456961 non-null  object 
 7   Listing Name           456756 non-null  object 
 8   Host ID                456961 non-null  int64  
 9   Host URL               456961 non-null  object 
 10  Host Name              456913 non-null  object 
 11  Host Since             456913 non-null  object 
 12  Host Response Time     398194 non-null  object 
 13  Host Response Rate     398194 non-null  object 
 14  Is Superhost           456913 non-null  o

### Aggregate dataset by Listing ID

Clean text From pancutations or undesired characters

In [172]:
# 1. Clean Text: Perform text cleaning, remove currency symbols & commas
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)
df['Host Response Rate'] = df['Host Response Rate'].replace('[\%,]', '', regex=True).astype(float)

# Fix Postal Code incorrect values, remove '\n' and other irrelevant text
df['Postal Code'] = df['Postal Code'].astype(str).str[:5]

Adjust Prices for Inflation

In [173]:
# Example CPI data
cpi_data = {
    2009: 92.2,
    2010: 93.2,
    2011: 95.2,
    2012: 97.1,
    2013: 98.5,
    2014: 99.4,
    2015: 100.0,
    2016: 100.5,
    2017: 102.0,
    2018: 103.8,
    2019: 105.3
}

# Base year for adjustment
base_year = 2019
base_cpi = cpi_data[base_year]

# Function to adjust price for inflation
def adjust_for_inflation(row):
    original_year = row['Review Date Year']
    original_price = row['Price']
    if original_year in cpi_data:
        original_cpi = cpi_data[original_year]
        adjusted_price = original_price * (base_cpi / original_cpi)
        return round(adjusted_price)
    else:
        return round(original_price)  # If year not in CPI data, return original price

df['review_date'] = pd.to_datetime(df['review_date'])
df['Review Date Year'] = df['review_date'].dt.year

# Apply the adjustment
df['Adjusted Price'] = df.apply(adjust_for_inflation, axis=1)

Aggregate the dataset by 'Listing ID'
- For numerical columns, we'll compute the mean
- For categorical columns, we'll take the first value (assuming consistency)

In [177]:
# Aggregate the dataset by 'Listing ID'
# For numerical columns, we'll compute the mean
# For categorical columns, we'll take the first value (assuming consistency)
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['number']).columns.difference(['Listing ID'])
aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
                                              **{col: 'first' for col in categorical_cols}})

# aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
#                                               **{col: lambda x: x.mode().iloc[0] if not x.mode().empty else None for col in categorical_cols}})

print(aggregated_df.shape)

(23536, 46)


### Features Selection
1. If a categorical column is not relevant to the analysis, we can remove it.
2. Listing URL, Listing Name, Host URL, Host Name: These are mostly unique to each listing, so not useful for category reduction

In [178]:
aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23536 entries, 2695 to 34682315
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Accomodates            23536 non-null  float64
 1   Accuracy Rating        18888 non-null  float64
 2   Adjusted Price         23536 non-null  float64
 3   Bathrooms              23507 non-null  float64
 4   Bedrooms               23516 non-null  float64
 5   Beds                   23501 non-null  float64
 6   Checkin Rating         18870 non-null  float64
 7   Cleanliness Rating     18892 non-null  float64
 8   Communication Rating   18886 non-null  float64
 9   Guests Included        23536 non-null  float64
 10  Host ID                23536 non-null  float64
 11  Host Response Rate     13046 non-null  float64
 12  Latitude               23536 non-null  float64
 13  Location Rating        18871 non-null  float64
 14  Longitude              23536 non-null  float64
 15  M

In [158]:
# drop the columns that is not helpful for prediction
df = aggregated_df.copy()
df = df.drop(columns=['Review ID', 'Reviewer ID', 'Reviewer Name', 'Listing URL','Listing Name',
                      'Host ID', 'Host URL', 'Host Name', 'City', 'Country Code', 'Country',
                      'First Review', 'Last Review', 'Square Feet', 'Business Travel Ready'])

### Reduce Large Categories

1. Group Rare Categories: If a categorical column has many unique values, we can group infrequent categories into an "Other" category like 'Reviewer Name'.
2. Merge Similar Categories: If there are similar categories (e.g., different spellings or formats of the same category), we can merge them.
3. Binning: For numerical categories (like "Overall Rating" or "Accommodates"), we can create bins to reduce the number of unique values.

In [179]:
# 2. Grouping neighbourhoods into Neighborhood Groups
if 'Neighborhood Group' in df.columns:
  neighbourhood_mapping = df.groupby('neighbourhood')['Neighborhood Group'].first()
  df['Neighbourhood Grouped'] = df['neighbourhood'].map(neighbourhood_mapping)

# 3. Reducing Property Types
property_mapping = {
    "Villa": "Vacation Rental",
    "Cottage": "Vacation Rental",
    "Bungalow": "Vacation Rental",
    "Cabin": "Vacation Rental",
    "Tiny house": "Vacation Rental",
    "Earth house": "Vacation Rental",
    "Treehouse": "Vacation Rental",
    "Hut": "Vacation Rental",
    "Barn": "Vacation Rental",
    "Houseboat": "Boats & Houseboats",
    "Boat": "Boats & Houseboats",
    "Camper/RV": "Mobile/Alternative Lodging",
    "Cave": "Mobile/Alternative Lodging",
    "Pension (South Korea)": "Mobile/Alternative Lodging",
    "Casa particular (Cuba)": "Mobile/Alternative Lodging",
}

# Apply mapping and assign 'Other' to rare categories
top_property_types = [
    "Apartment", "Loft", "House", "Townhouse", "Condominium", "Serviced apartment",
    "Hotel", "Hostel", "Guesthouse", "Bed and breakfast", "Boutique hotel"
]

df['Property Type Reduced'] = df['Property Type'].apply(
    lambda x: property_mapping.get(x, x) if x in top_property_types or x in property_mapping else "Other"
)

# 4. Binning Postal Codes (first two digits represent broad area)
df['Postal Code Reduced'] = df['Postal Code'].astype(str).str[:2]  # Use only first 2 digits

### Transform/Manipulate data

In [180]:
# Extracting years from date columns
df['Host Since'] = pd.to_datetime(df['Host Since'])
df['Host Since Year'] = df['Host Since'].dt.year

# 5. transform true/false into bool
df['Instant Bookable'] = df['Instant Bookable'].replace({'t': True, 'f': False})
df['Is Superhost'] = df['Is Superhost'].replace({'t': True, 'f': False})
df['Is Exact Location'] = df['Is Exact Location'].replace({'t': True, 'f': False})
df['Instant Bookable'] = df['Instant Bookable'].replace({'t': True, 'f': False})

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23536 entries, 2695 to 34682315
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Accomodates            23536 non-null  float64       
 1   Accuracy Rating        18888 non-null  float64       
 2   Adjusted Price         23536 non-null  float64       
 3   Bathrooms              23507 non-null  float64       
 4   Bedrooms               23516 non-null  float64       
 5   Beds                   23501 non-null  float64       
 6   Checkin Rating         18870 non-null  float64       
 7   Cleanliness Rating     18892 non-null  float64       
 8   Communication Rating   18886 non-null  float64       
 9   Guests Included        23536 non-null  float64       
 10  Host Response Rate     13046 non-null  float64       
 11  Latitude               23536 non-null  float64       
 12  Location Rating        18871 non-null  float64       
 13  

In [181]:
df.head(2)

,Review ID,review_date,Reviewer ID,Reviewer Name,Comments,Listing ID,Listing URL,Listing Name,Host ID,Host URL,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,City,Postal Code,Country Code,Country,Latitude,Longitude,Is Exact Location,Property Type,Room Type,Accomodates,Bathrooms,Bedrooms,Beds,Square Feet,Price,Guests Included,Min Nights,Reviews,First Review,Last Review,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Review Date Year,Adjusted Price,Neighbourhood Grouped,Property Type Reduced,Postal Code Reduced,Host Since Year
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,58660447.0,2016-01-03,14876562.0,Dennis,"Very nice, clean appartement! Josh was really ...",10002699,https://www.airbnb.com/rooms/10002699,Trendy and peaceful KreuzkÃ¶lln,44696672,https://www.airbnb.com/users/show/44696672,Josh,2015-09-20,NaN,NaN,False,NeukÃ¶lln,NeukÃ¶lln,Berlin,12047,DE,Germany,52.49234,13.42994,True,Apartment,Entire home/apt,4,1.0,2.0,2.0,NaN,60.0,1,1,1,01-03-16,01-03-16,100.0,8.0,10.0,6.0,10.0,10.0,10.0,False,f,2016.0,63,NeukÃ¶lln,Apartment,12,2015.0
1,NaN,NaT,NaN,NaN,NaN,10002922,https://www.airbnb.com/rooms/10002922,Sehr zentrale Wohnung in Berlin,45848709,https://www.airbnb.com/users/show/45848709,Belinda,2015-10-05,NaN,NaN,False,Prenzlauer Berg,Pankow,Berlin,10405,DE,Germany,52.53292,13.41502,False,Apartment,Entire home/apt,2,1.0,1.0,1.0,NaN,52.0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,f,NaN,52,Pankow,Apartment,10,2015.0


### EDA (Exploratory Data Analysis)

In [184]:
df_EDA = df.drop(columns=['Host Since', 'neighbourhood', 'Property Type', 'Postal Code'])
# 'Comments','Instant Bookable', 'Is Superhost', 'Is Exact Location'

Store EDA deady dataset as pkl file

In [188]:
import pickle
with open('df_EDA.pkl', 'wb') as f:
  pickle.dump(df_EDA, f)